In [3]:
!pip install fastbook
from fastbook import *
from fastai.vision.all import *
import numpy as np
import PIL
from os import *
from fastai import *
from fastai.tabular import *
from fastai.vision import *
import fastai.vision as vision
#from fastai.vision.widgets import *
#from utils import *

In [4]:
#quick test to see an image
pictu = "../input/smalldibas/cropped/Pseudomonas.aeruginosa/img7094.png"
dest = Path(pictu)
im = Image.open(dest)
im.to_thumb(2048,1532)

In [5]:
path = r"../input/smalldibas/cropped"

#creating data block
dls = DataBlock(
    blocks=(ImageBlock,CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
).dataloaders(path)

dls.show_batch(max_n=6)

In [ ]:
##loading my pretrained pth didnt work bc apparently it cant find the file on kaggle?
#Altlearn = vision_learner(dls, resnet50, metrics=error_rate)
#Altlearn.load(Path("../input/badbacteriamodel/OriginalModel"))

In [6]:
learn = vision_learner(dls, resnet50, metrics=error_rate)
learn.fine_tune(4)

In [8]:
learn.lr_find()

In [73]:
##cleaning bad images (bc my imput will definetely be of the same quality)
##didnt work lol theres some definition issues
#cleaner = ImageClassifierCleaner(learn)
#cleaner

In [12]:
#retraining w different learning rate at the loss valley
learn.fit_one_cycle(10, 5e-5) #cbs=[ShowGraphCallback()]

In [ ]:
Model evaluation

In [14]:
interp = ClassificationInterpretation.from_learner(learn)

In [15]:
interp.plot_confusion_matrix(figsize=(30,30), dpi=100)

In [19]:
interp.plot_top_losses(6, figsize=(24,24))

It can be seen here the model seems to do surprisingly well, and the images it struggles on are either almost blank or the strains are extremely similar phenotypically. 

In [22]:
#functions for trying out the initial model on my images
def identifyBacteria(impath):
    is_bacteria,_,probs = learn.predict(PILImage.create(impath))
    print(f"This is a: {is_bacteria}.")
    print(f"Probability: {maximum(probs)}")

In [30]:
testing = "../input/smalldibas/cropped/Lactobacillus.rhamnosus/img15286.png"
learn.predict(PILImage.create(testing))

In [31]:
#exporting
learn.export()